In [61]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer

In [52]:
data = pd.read_csv('titanic.csv')
data.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [53]:

original_train = train.copy() 
original_test = test.copy()

full_data = [data]

data['Has_Cabin'] = data["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
#test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

#SibSp nombre de freres et soeurs/Epouses ; Parch - Number of Parents/Children Aboard;
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

# for dataset in full_data:
#     dataset['Embarked'] = dataset['Embarked'].fillna('S')

# for dataset in full_data:
#     #fare = tarif
#     dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())

# # suppression des NULL dans column Age
# for dataset in full_data:
#     age_mean = dataset['Age'].mean()
#     age_std = dataset['Age'].std() #Ecart Type
#     print(age_std)
#     age_null_count = dataset['Age'].isnull().sum()
#     age_null_random_list = np.random.randint(age_mean - age_std, age_mean + age_std, size=age_null_count)
#     dataset.loc[np.isnan(dataset['Age']), 'Age'] = age_null_random_list
#     dataset['Age'] = dataset['Age'].astype(int)

# Fonction pour avoir les titre
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # Si le titre Existe le returneeeeeeeee.
    if title_search:
        return title_search.group(1)
    return ""

for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Grouper tout les titre rare en un seul "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

# for dataset in full_data:
#     dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
#     #title_mapping = {"Mr": 1, "Master": 2, "Mrs": 3, "Miss": 4, "Rare": 5}
    #dataset['Title'] = dataset['Title'].map(title_mapping)
    #dataset['Title'] = dataset['Title'].fillna(0)
#Embarked - Port d'Embarquement (C = Cherbourg; Q = Queenstown; S = Southampton)
    #dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [54]:
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin']
data = data.drop(drop_elements, axis = 1)

In [55]:
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Has_Cabin,FamilySize,IsAlone,Title
0,0,3,male,22.0,1,0,7.2500,S,0,2,0,Mr
1,1,1,female,38.0,1,0,71.2833,C,1,2,0,Mrs
2,1,3,female,26.0,0,0,7.9250,S,0,1,1,Miss
3,1,1,female,35.0,1,0,53.1000,S,1,2,0,Mrs
4,0,3,male,35.0,0,0,8.0500,S,0,1,1,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,0,1,1,Rare
887,1,1,female,19.0,0,0,30.0000,S,1,1,1,Miss
888,0,3,female,NaN,1,2,23.4500,S,0,4,0,Miss
889,1,1,male,26.0,0,0,30.0000,C,1,1,1,Mr


In [56]:
numerical = ['Pclass', 'Age', 'Fare','FamilySize']
categorical= ['Sex', 'Embarked', 'Title',]

In [62]:
numerical_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'), StandardScaler())
categorical_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder())

In [63]:
preprocessor = make_column_transformer((numerical_pipeline,numerical),(categorical_pipeline,categorical))

In [64]:
model = make_pipeline(preprocessor, DecisionTreeClassifier())
params = {'decisiontreeclassifier__min_samples_split': [2,20],
            'decisiontreeclassifier__max_depth' : [1,50]
         }

grid = GridSearchCV(model, param_grid = params, cv=4)


In [67]:
X = data.drop(['Survived'], axis=1)
y = data['Survived']

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=0)

In [69]:
grid.fit(X_train,y_train)

C:\Users\CISSE Issa\anaconda3\lib\site-packages\sklearn\impute\_base.py:49: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(array)
C:\Users\CISSE Issa\anaconda3\lib\site-packages\sklearn\impute\_base.py:49: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warnin

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         ['Pclass',
                                                                          'Age',
                                                                          'Fare',
                                                                          'FamilySize']),
                                      

In [70]:
grid.best_params_

{'decisiontreeclassifier__max_depth': 50,
 'decisiontreeclassifier__min_samples_split': 20}

In [71]:
grid.score(X_test,y_test)

0.8340807174887892

In [73]:
tree.plot_tree(model)
plt.show()

NotFittedError: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.